
# Day 7: No Space Left On Device ---

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:
```
$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?
```
You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:
```
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
```
The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
cd / switches the current directory to the outermost directory, /.
ls means list. It prints out all of the files and directories immediately contained by the current directory:
123 abc means that the current directory contains a file named abc with size 123.
dir xyz means that the current directory contains a directory named xyz.
Given the commands and output in the example above, you can determine that the filesystem looks visually like this:
```
- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
```
Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.

The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).

Directory d has total size 24933642.

As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?


## Approach

Well this is bringing back bad memories of running out of hard drive space!

Initial thoughts:
 - going to need to build up a tree structure
 - going to need to sum up the nodes of the tree
 - building the tree is going to be weird

Tree structure in python? Probably going to be a dictionary.
Summing up trees... some sort of fold operation.

Looks like we're going to need to build our own data structures!

In [52]:
#Python seems to struggle with a parameter having the type of the class it's declaring - seems reasonable.
#But it worked before, suggesting that it's hackable.

class Directory:
    pass


class Directory:
    def __init__(self, name:str='/', parent:Directory=None):
        self.name = name
        self.childDirectories:{str:Directory} = {}
        self.files:{str:int} = {}
        self.parent = parent

    #print a pretty tree output
    def __str__(self):
        output = ''
        output += '- ' + self.name + ' (dir)' + '\n'
        for n,d in self.childDirectories.items():
            #output += str(d) + '\n'
            #left pad the output
            for l in str(d).splitlines():
                output += '  ' + l + '\n'
        for f,s in self.files.items():
            output += '  - ' + f + ' (file, size=' + str(s) +')' + '\n'
        return output

    #calculate size
    def size(self)->int:
        total = 0
        for n,d in self.childDirectories.items():
            total += d.size()
        for f,s in self.files.items():
            total += s
        return total
    
    def findSmallDirectories(self, maxSize:int):
        #BDirectory names are not unqiue... so should just use an array 
        output:[int] = []

        #include self
        mysize = self.size()
        if mysize < maxSize:
            output.append(mysize)

        for n,d in self.childDirectories.items():
            output.extend(d.findSmallDirectories(maxSize))
        return output
    
    def findBigDirectories(self, minSize:int):
        output:[int] = []

        #include self
        mysize = self.size()
        if mysize > minSize:
            output.append(mysize)

        for n,d in self.childDirectories.items():
            output.extend(d.findBigDirectories(minSize))
        return output

In [53]:
testData = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

In [54]:
def buildTree(input:str)->Directory:
    root = Directory()
    currentDir = root

    for l in input.splitlines():
        #print(l)
        if l[0] == '$':
            #print('command detected')
            if l == '$ cd /':
               currentDir = root
            elif l[:4] == '$ cd':
                toDir = l[5:] #chop off first 5 characters
                if toDir == '..':
                    currentDir = currentDir.parent
                else:
                    currentDir = currentDir.childDirectories[toDir] #not handling error of directory not found
            #other command only going to be ls, so can ignore?? (assuming the input only traverses tree once!)
        else:
            #output of ls command
            if l[:3] == 'dir':
                dirName = l[4:]
                newDir = Directory(dirName, currentDir)
                currentDir.childDirectories[dirName] = newDir
            else:
                fileComponents = l.split()
                currentDir.files[fileComponents[1]] = int(fileComponents[0])
    return root

#test
print('----- tests ------')
tree = buildTree(testData)
print(tree)

----- tests ------
- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)



Test output should look like:
```
- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
```

Now need to calculate directory sizes, including sub-directories.
We can re-use the __str__ function approach of recurrsion to achieve this. So let's go back and add that to the Directory class.


In [55]:
#test size
print(tree.size())
# should be 48381165

48381165


So, to the quiz question:

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?


Approach: again, going to be easier to add to a method of Directory.




In [56]:
smallDir = tree.findSmallDirectories(100000)
print(smallDir)
print(sum(smallDir))


[94853, 584]
95437


In [57]:
#actual run
input = open("day7input.txt").read()
root = buildTree(input)
print(root)

print('--- full tree size ---')
print(root.size())
smallDir = root.findSmallDirectories(100000)
print(smallDir)
print(sum(smallDir))


- / (dir)
  - cmjgvh (dir)
    - hdh (dir)
      - htplc.gdw (file, size=267125)
    - hgrpfmt (dir)
      - lndwz (file, size=39132)
      - rffmsvdw (file, size=280595)
    - sjgvbd (dir)
      - ghg.zmq (file, size=26464)
      - zsgdbd.dmm (file, size=1533)
    - hdh.sjv (file, size=134565)
    - mjtq.ffd (file, size=282147)
    - rvmzv.rtb (file, size=42343)
    - wgtjmb.thf (file, size=31468)
  - czrzl (dir)
    - cmh (dir)
      - nglsj (dir)
        - cjvph.fvc (file, size=307082)
        - szs.vpj (file, size=286825)
      - szs (dir)
        - fgpnnvm (file, size=6233)
        - jjc.szq (file, size=94070)
      - vdpqhvrm.mcz (file, size=76775)
    - hpgnd.mmt (file, size=242795)
    - hps.ptg (file, size=157748)
    - qjrhbjql.zdc (file, size=129797)
    - sfhrzzcd.hwm (file, size=18290)
    - sngg.vdw (file, size=63141)
  - fcbt (dir)
    - grncq.brq (file, size=150417)
  - hdh (dir)
    - cljdmh (dir)
      - blsrsqz (dir)
        - dvqsffcn (file, size=275130)
      - grn

# Part Two ---

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

In the example above, the total size of the outermost directory (and thus the total amount of used space) is 48381165; this means that the size of the unused space must currently be 21618835, which isn't quite the 30000000 required by the update. Therefore, the update still requires a directory with total size of at least 8381165 to be deleted before it can run.

To achieve this, you have the following options:

- Delete directory e, which would increase unused space by 584.
- Delete directory a, which would increase unused space by 94853.
- Delete directory d, which would increase unused space by 24933642.
- Delete directory /, which would increase unused space by 48381165.
- Directories e and a are both too small; deleting them would not free up enough space. However, directories d and / are both big enough! Between these, choose the smallest: d, increasing unused space by 24933642.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

## Approach
Extend Directory class to produce an array of all directory sizes
Filter list to remove items below a certain size - actually, do this is the Directory method
Find the smallest element - note, don't need to know directory name, just size.



In [58]:
print('--- test data ---')
print(tree.size())
bigDir = tree.findBigDirectories(8381165)
print(bigDir)
smallestBiggest = min(bigDir)
print(smallestBiggest)



--- test data ---
48381165
[48381165, 24933642]
24933642


In [59]:
#real data
totalDiskSpace = 70000000
spaceNeeded = 30000000
freeSpace = totalDiskSpace - root.size()
needToDelete = spaceNeeded - freeSpace
print(freeSpace)
print(needToDelete)

bigDir = root.findBigDirectories(needToDelete)
print(bigDir)
smallestBiggest = min(bigDir)
print(smallestBiggest)

24825975
5174025
[45174025, 10851370, 9665817, 5649896, 27642694, 25773269, 11694215, 9510609, 6824953, 6175618]
5649896
